In [0]:
import os
import atexit
import sys

import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
import findspark
from sparkhpc import sparkjob

#Exit handler to clean up the Spark cluster if the script exits or crashes
def exitHandler(sj,sc):
    try:
        print('Trapped Exit cleaning up Spark Context')
        sc.stop()
    except:
        pass
    try:
        print('Trapped Exit cleaning up Spark Job')
        sj.stop()
    except:
        pass

findspark.init()

#Parameters for the Spark cluster
nodes=1
tasks_per_node=12
memory_per_task=1024 #1 gig per process, adjust accordingly
# Please estimate walltime carefully to keep unused Spark clusters from sitting 
# idle so that others may use the resources.
walltime="24:00" #1 hour
os.environ['SBATCH_PARTITION']='parallel' #Set the appropriate ARC partition

sj = sparkjob.sparkjob(
     ncores=nodes*tasks_per_node,
     cores_per_executor=tasks_per_node,
     memory_per_core=memory_per_task,
     walltime=walltime
    )

sj.wait_to_start()
sc = sj.start_spark()

#Register the exit handler                                                                                                     
atexit.register(exitHandler,sj,sc)

#You need this line if you want to use SparkSQL
sqlCtx=SQLContext(sc)

INFO:sparkhpc.sparkjob:Submitted batch job 2616529

INFO:sparkhpc.sparkjob:Submitted cluster 0


In [0]:
df = sqlCtx.read.csv('CleanedNews.csv/part-00000-e0c20413-d9a2-4ae3-bc41-a77b460c6a58-c000.csv',inferSchema=True)
df = df.withColumnRenamed('_c0','claim').withColumnRenamed('_c1','claimant').withColumnRenamed('_c2','articles').withColumnRenamed('_c3','label')
df.printSchema()

root
 |-- claim: string (nullable = true)
 |-- claimant: string (nullable = true)
 |-- articles: string (nullable = true)
 |-- label: integer (nullable = true)



In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StandardScaler,VectorAssembler
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol='articles',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ss = StandardScaler(inputCol='tf_idf',outputCol='scaled')
assembler = VectorAssembler(inputCols=['scaled'],outputCol='features')

pipe = Pipeline(stages=[tokenizer,stop_remove,count_vec,idf,ss,assembler])
pipelineFit = pipe.fit(df)
dataset = pipelineFit.transform(df)

In [0]:
training,test = dataset.randomSplit(weights = [0.8,0.2],seed = 0 )

In [0]:
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes()

gridSearch = ParamGridBuilder().addGrid(nb.smoothing,[0.0,0.2,0.4,0.6,0.8,1.0]).build()
cvEvaluater = MulticlassClassificationEvaluator(metricName="weightedPrecision",predictionCol="prediction")

cv = CrossValidator(estimator=nb,estimatorParamMaps=gridSearch,evaluator=cvEvaluater)
cvModel = cv.fit(training)

In [0]:
cvModel.avgMetrics

[0.5495986718141771,
 0.5627333232587854,
 0.565204943914567,
 0.5670082755380683,
 0.5682727872583998,
 0.5704466300774521]

In [0]:
from sklearn.metrics import classification_report
prediction = cvModel.transform(test)
y_true = prediction.select('label').collect()
y_pred = prediction.select('prediction').collect()
print (classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1500
           1       0.57      0.56      0.57      1281
           2       0.15      0.16      0.15       341

    accuracy                           0.56      3122
   macro avg       0.45      0.45      0.45      3122
weighted avg       0.56      0.56      0.56      3122

